In [16]:
# check the performance benchmarks between two versions
import os, sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", "gtracr"))

from trajectory import Trajectory
from constants import EARTH_RADIUS

import time

In [ ]:
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=5.0)

In [ ]:
# should be in Monte carlo in the future
traj_dip = Trajectory(
        "p+",
        latitude=0.,
        longitude=0.,
        detector_altitude=100.,
        zenith_angle=0.,
        azimuth_angle=0.,
        particle_altitude=100.,
        rigidity=40.,
        bfield_type="dipole"
    )

traj_igrf = Trajectory(
        "p+",
        latitude=0.,
        longitude=0.,
        detector_altitude=100.,
        zenith_angle=0.,
        azimuth_angle=0.,
        particle_altitude=100.,
        rigidity=40.,
        bfield_type="igrf"
    )

In [ ]:
max_iter = 100    # maximum number of iterations
dt = 1e-5    # step size in integration
max_step = 10000   # max steps in integration
# initialize performance benchmark time variables
time_python_dip = 0.
time_cpp_novec_dip = 0.
time_cpp_vec_dip = 0.
time_python_igrf = 0.
time_cpp_novec_igrf = 0.
time_cpp_vec_igrf = 0.
time_hydra = 0.

In [ ]:
for i in range(max_iter):
    # evaluate for python version
    start_time_python = time.time()
    trajdata_python = traj_dip.get_trajectory(dt=dt, max_step=max_step, use_python=True)  
    stop_time_python = time.time()
    time_python_dip += stop_time_python - start_time_python
    # evaluate for the C++ version unvectorized
    start_time_cpp_novec = time.time()
    trajdata_cpp_novec = traj_dip.get_trajectory(dt=dt, max_step=max_step, use_unvectorized=True)  
    stop_time_cpp_novec = time.time()
    time_cpp_novec_dip += stop_time_cpp_novec - start_time_cpp_novec
    # evaluate for the C++ version vectorized
    start_time_cpp_vec = time.time()
    trajdata_cpp_vec = traj_dip.get_trajectory(dt=dt, max_step=max_step)  
    stop_time_cpp_vec = time.time()
    time_cpp_vec_dip += stop_time_cpp_vec - start_time_cpp_vec

In [ ]:
# get the average values for each of the results
avg_timearr_dip = np.array([time_python_dip, time_cpp_novec_dip, time_cpp_vec_dip]) / max_iter
avg_timearr_dip = np.array([time_cpp_novec_dip, time_cpp_vec_dip]) / max_iter
rel_avgtime_dip = avg_timearr_dip / avg_timearr_dip[-1]

In [ ]:
 # the labels, needs to be modified every time different modification is used
labels = ["Python (Vector)", "C++ (non-Vector)", "C++ (Vector)"] 
labels = ["C++ (non-Vector)", "C++ (Vector)"] 
label_pos = np.arange(len(labels))
width = 0.35
# plot the histogram comparing the two values
fig, ax = plt.subplots(figsize=(12,9))
fig.patch.set_facecolor('white')  # to make saved plot have all white bg
bar = ax.bar(label_pos, avg_timearr_dip, width)
# ax.bar(label_pos, rel_avgtime_dip, width)
ax.set_xticks(label_pos)
ax.set_xticklabels(labels)
ax.set_ylabel("Time [s]")
# ax.set_ylabel("Scale Factor relative to {:s} result".format(labels[-1]))
ax.set_title("Performance benchmark of trajectory calculation with dipole magnetic field for {:d} iterations".format(max_iter))

autolabels(bar)  # add labels at top of bar chart

PLOT_DIR = os.path.join(os.getcwd(), "..", "..", "..", "gtracr_plots")

savefig_labels = ["pyvec", "cpp_novec", "cpp_vec"]
plt.savefig(os.path.join(PLOT_DIR, "traj_performance_benchmark.png"), bbox_inches='tight')